In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import eda

from tqdm import tqdm

# os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id']

In [2]:
def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
train = elo_load_data('train')
test = elo_load_data('test')

100%|██████████| 3/3 [00:00<00:00, 183.80it/s]


In [3]:
from dateutil.parser import parse
def diff_of_days(day1, day2):
    try:
        days = (parse(day1) - parse(day2)).days
    except TypeError:
        return np.nan
    return days
train['term_days_from_first@'] = train['first_active_month'].map(lambda x: diff_of_days('2018-02-01', x))
test['term_days_from_first@'] = test['first_active_month'].map(lambda x: diff_of_days('2018-02-01', x))

In [4]:
from joblib import Parallel, delayed
new = elo_load_data('new')[[key, 'purchase_date']]
new = new.groupby(key)['purchase_date'].max().reset_index()
df = pd.concat([train, test], axis=0)
df = df.merge(new, how='left', on=key)

def diff_of_days(args):
    uid = args[0]
    day1 = args[1]
    day2 = args[2]
    try:
        days = (parse(day1) - parse(day2)).days
        return [uid, days]
    except TypeError:
        return [uid, np.nan]
id_list = df[key].values
first = df['first_active_month'].values
last = df['purchase_date'].values
p_list = Parallel(n_jobs=-1)([delayed(diff_of_days)(args) for args in zip(id_list, last, first)])
df_diff = pd.DataFrame(np.array(p_list), columns=[key, 'term_days_range_first_last@'])

100%|██████████| 3/3 [00:00<00:00,  3.40it/s]


In [6]:
feat_no = '104_tra_'
train_idx = train[key].values
test_idx = test[key].values
# np.nanと整数のみだとobjectになってしまう
df_diff['term_days_range_first_last@'] = df_diff['term_days_range_first_last@'].astype('float32')
result = df.merge(df_diff, how='inner', on=key)
for col in result.columns:
    if col.count('feature') or col.count('@'):
        tmp_train = result.set_index(key).loc[train_idx, col].values
        tmp_test = result.set_index(key).loc[test_idx, col].values 
#         utils.to_pkl_gzip(obj=train[col].values, path=f'../features/4_winner/{feat_no}train_{col}@')
#         utils.to_pkl_gzip(obj=test[col].values, path=f'../features/4_winner/{feat_no}test_{col}@')
        utils.to_pkl_gzip(obj=tmp_train,  path=f'../features/4_winner/{feat_no}train_{col}@')
        utils.to_pkl_gzip(obj=tmp_test, path=f'../features/4_winner/{feat_no}test_{col}@')